# A TensorFlow model for classifying MNIST dataset.

First, import TensorFlow and read MNIST datasets.

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


Add placeholders.

In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

First layer.

In [7]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01), name="W1")
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

Second layer.

In [8]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01), name='W2')
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

Third layer.

In [9]:
W3 = tf.Variable(tf.random_normal([7 ** 2 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 ** 2 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

Final layer.

In [10]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

Initialize the network.

In [12]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Train the network. Go get a coffee. If CPU, get several.

In [14]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
total_epoch = 15 # adjust this

for epoch in range(total_epoch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        total_cost += cost_val

    print('Epoch #{0}, Cost: {1}'.format(epoch, total_cost / total_batch))

print('Training complete!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('Accuracy: {0}'.format(sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                                           Y: mnist.test.labels,
                                                           keep_prob: 1})))

Epoch #0, Cost: 0.3407987456904216
Epoch #1, Cost: 0.10619414171542634
Epoch #2, Cost: 0.07693003678355705
Epoch #3, Cost: 0.06111118069680577
Epoch #4, Cost: 0.04890983201478693
Epoch #5, Cost: 0.042774163527117874
Epoch #6, Cost: 0.03424722978252579
Epoch #7, Cost: 0.031026833901202984
Epoch #8, Cost: 0.027507262400520795
Epoch #9, Cost: 0.0249465579518371
Epoch #10, Cost: 0.021685813034630633
Epoch #11, Cost: 0.018830497651784257
Epoch #12, Cost: 0.01845069070272571
Epoch #13, Cost: 0.016472498943515927
Epoch #14, Cost: 0.015194867078905439
Training complete!
Accuracy: 0.9904999732971191
